# ICESAT-2 AWS云数据访问
此笔记本（{nb-download}`dlownload <is2_cloud_data_access.ipynb>`）说明了使用Icepyx访问通过AWS（Amazon Web Services）us-West2 Hub S3数据存储库当前可用的ICEPYX的使用。

## 笔记
1. ICESAT-2数据于2022年9月29日在云上公开可用。因此，NSIDC仍在开发访问方法和示例工作流程，并且现在需要更新Icepyx中的基础代码（这些数据（以及关联的数据）可以使用元数据）。感谢您在此过渡期间的耐心和贡献（例如报告错误，分享您的代码等）！
2.此示例及其描述的代码是正在进行的开发的一部分。在整个示例中描述了使用这些功能的当前限制。
3.您**必须**在AWS实例中工作。否则，您将获得权限错误。

## Querying for data and finding s3 urls

In [ ]:
import icepyx as ipx

In [ ]:
# Make sure the user sees important warnings if they try to read a lot of data from the cloud
import warnings
warnings.filterwarnings("always")

我们将通过创建Icepyx查询对象来启动我们经常做的方式。

In [ ]:
short_name = 'ATL03'
spatial_extent = [-45, 58, -35, 75]
date_range = ['2019-11-30','2019-11-30']

In [ ]:
reg = ipx.Query(short_name, spatial_extent, date_range)

### 获取颗粒S3 URL

使用此查询对象，您可以获取可用颗粒的列表。此功能返回包含颗粒ID列表和相应URL列表的列表。使用`cloud = true`获取所需的S3 URL。

In [ ]:
gran_ids = reg.avail_granules(ids=True, cloud=True)
gran_ids

## 确定感兴趣的变量

有几种查看可用变量的方法。一种是使用现有查询对象：

In [ ]:
reg.order_vars.avail()

Another way is to use the variables module:

In [ ]:
ipx.Variables(product=short_name).avail()

我们还可以使用查询对象的特定S3 filepath来执行此操作：

In [ ]:
ipx.Variables(path=gran_ids[1][0]).avail()

从这些方法中的任何一种中，我们都可以看到“ H_PH”是此数据产品的变量，因此我们将在下一步中读取该变量。

#### 使用S3 URL列出变量的注释

我们可以将变量模块与S3 URL一起使用，以与本地文件相同的方式探索可用的数据变量。但是，一个重要的区别是如何创建可用变量列表。读取本地文件时，变量模块将遍历整个文件，并搜索该文件中存在的变量。此方法使用S3数据太耗时了，因此可以从文件中读取数据产品的产品 /版本，并且与该产品 /版本关联的所有可能的变量都是可用的。只要您使用NSIDC提供的NSIDC提供了通过EarthData搜索提供的S3路径，并且查询对象将相同。

#### 认证注释

请注意，访问云数据需要两层身份验证：1）使用您的EarthData登录2）对云访问进行身份验证。这两者都发生在幕后，而无需用户提供任何明确的命令。

Icepyx使用EarthAccess生成您的S3数据访问令牌，这将在 *一个小时内有效。一个小时后，Icepyx还将为您续签令牌，因此，如果在几个小时内查看令牌，您可能会注意到该值将会改变。

如果您确实想查看S3凭据，则可以使用以下方式访问它们：

In [ ]:
# uncommenting the line below will print your temporary aws login credentials
# reg.s3login_credentials

````{invernition}重要身份验证更新
以前，Icepyx要求您明确使用`.earthdata_login（）`函数登录。运行此功能被弃用并会导致错误，因为Icepyx会根据需要调用登录功能。用户仍然需要提供其凭据。
````

## 选择一个数据文件并访问数据

**注意：如果尝试阅读数据时会遇到有权限的错误，则可能不会从US-West2中的AWS Hub发送请求。我们目前正在研究如何提醒用户，如果由于这个原因，他们将无法访问云中的ICESAT-2数据**

我们准备阅读我们的数据！我们通过创建读取对象并使用从查询对象返回的S3 URL来做到这一点。

In [ ]:
# the first index, [1], gets us into the list of s3 urls
# the second index, [0], gets us the first entry in that list.
s3url = gran_ids[1][0]
# s3url =  's3://nsidc-cumulus-prod-protected/ATLAS/ATL03/004/2019/11/30/ATL03_20191130221008_09930503_004_01.h5'

Create the Read object

In [ ]:
reader = ipx.Read(s3url)

This reader object gives us yet another way to view available variables.

In [ ]:
reader.vars.avail()

Next, we append our desired variable to the `wanted_vars` list:

In [ ]:
reader.vars.append(var_list=['h_ph'])

Finally, we load the data

In [ ]:
%%time

# This may take 5-10 minutes
reader.load()

### 一些重要的警告

虽然云数据读数在Icepyx中起作用，但它非常慢。近似正时表明，每个文件中每个文件的加载时间约为6分钟。因此，如果您尝试一次加载三个以上的变量或两个文件，您将收到警告。

缓慢的负载速度是使使云数据可行的许多步骤的演示 - 数据供应链需要优化的源数据，有效的低级数据读取器以及能够使用最快的低级数据读取器的高级库。并非所有这些作品现在都完全开发了，但是进步使它令人兴奋，并且有很多贡献的空间！

#### Credits
* notebook by: Jessica Scheick and Rachel Wegener